## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
#city_data_df.head()

In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?: "))
max_temp = float(input("What is the maximum temperature you would like for your trip?: "))

What is the minimum temperature you would like for your trip?: 75
What is the maximum temperature you would like for your trip?: 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Georgetown,MY,5.4112,100.3354,79.90,86,73,4.56,broken clouds
4,4,Montecristo,CO,8.2971,-74.4733,84.97,71,84,3.76,broken clouds
5,5,Campos Novos,BR,-27.4017,-51.2250,79.90,46,32,6.60,scattered clouds
8,8,Lucea,JM,18.4510,-78.1736,82.85,69,100,5.14,overcast clouds
13,13,The Valley,AI,18.2170,-63.0578,84.09,74,40,11.50,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Georgetown,MY,79.90,broken clouds,5.4112,100.3354,
4,Montecristo,CO,84.97,broken clouds,8.2971,-74.4733,
5,Campos Novos,BR,79.90,scattered clouds,-27.4017,-51.2250,
8,Lucea,JM,82.85,overcast clouds,18.4510,-78.1736,
13,The Valley,AI,84.09,scattered clouds,18.2170,-63.0578,
19,Saint-Philippe,RE,76.59,overcast clouds,-21.3585,55.7679,
21,Morondava,MG,80.64,few clouds,-20.2833,44.2833,
22,Cape Town,ZA,76.26,scattered clouds,-33.9258,18.4232,
26,Mahebourg,MU,79.25,overcast clouds,-20.4081,57.7000,
28,Axim,GH,83.28,overcast clouds,4.8699,-2.2405,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found...skipping...")
        

Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...
Hotel not found...skipping...


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace("", np.nan, regex=True)
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))